In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [37]:
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath, encoding='latin1')
    data['Actors'] = data[['Actor 1', 'Actor 2', 'Actor 3']].fillna('').agg(', '.join, axis=1)
    data.drop(['Actor 1', 'Actor 2', 'Actor 3'], axis=1, inplace=True)
    data.dropna(subset=['Rating'], inplace=True)
    return data

In [38]:
def build_pipeline():
    # Define preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('genre', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['Genre']),
            ('director', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['Director']),
            ('actors', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['Actors'])
        ],
        remainder='passthrough'
    )
    # Build the pipeline with Ridge regression
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', StandardScaler()),  # Scaling the data
        ('model', Ridge(alpha=1.0))    # Use Ridge regression for regularization
    ])

    return pipeline

In [39]:
def main(filepath):
    data = load_and_preprocess_data(filepath)

In [40]:
    X = data[['Genre', 'Director', 'Actors']]
    y = data['Rating']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Genre']),
                                                 ('director',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Director']),
                                                 ('actors',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Actors'])])),
                ('scaler', StandardScaler()), ('model', Ridge())])

In [43]:
y_pred = pipeline.predict(X_test)

In [44]:
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 1.99


In [45]:
    example_movie = pd.DataFrame({
        'Genre': ['Action'],
        'Director': ['John Doe'],
        'Actors': ['Actor A, Actor B']
    })
    predicted_rating = pipeline.predict(example_movie)
    print(f'Predicted Rating: {predicted_rating[0]:.2f}')

Predicted Rating: 3.38


In [46]:
if __name__ == "__main__":
    main('/content/IMDb Movies India.csv')